In [ ]:
!pip3 install kafka-python

In [1]:
import pandas as pd
import ccxt.pro as ccxtpro
from asyncio import run
from kafka import KafkaProducer
import asyncio
import time
import json
import sys
from snowflake.connector import connect
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
# sys.path.append("")

In [2]:
conn1 = connect(account= 'bta69289.us-east-1',
                user = 'geetanjali',
                password = 'Token@1234',
                database_name = 'DEMO_DB',
                schema_name = 'PUBLIC',
                role='TM_INTERNAL_TEAM')

In [ ]:
# producer = KafkaProducer(
#     security_protocol='SASL_SSL',
#     sasl_mechanism = 'AWS_MSK_IAM',
#     api_version=(0,11,5),
#     bootstrap_servers=[
#         "b-1.tmccxtdata.022mgq.c5.kafka.us-east-1.amazonaws.com:9098",
#         "b-3.tmccxtdata.022mgq.c5.kafka.us-east-1.amazonaws.com:9098",
#         "b-2.tmccxtdata.022mgq.c5.kafka.us-east-1.amazonaws.com:9098"
#     ]
# )


In [14]:
query1="""select * from CRYPTO_DB.CCXT_PRO.CCXT_TOKEN_MAP where EXCHANGE='ascendex' and QUOTESYMBOL in('USD','USDT') and 
BASESYMBOL not in (select distinct(T1.BASESYMBOL)
  from CRYPTO_DB.CCXT_PRO.CCXT_TOKEN_MAP as T1 inner join CRYPTO_DB.CCXT_PRO.CCXT_TOKEN_MAP as T2
 on T1.BASESYMBOL=T2.BASESYMBOL and T1.EXCHANGE=T1.EXCHANGE where T1.EXCHANGE='ascendex' and T1.QUOTESYMBOL!=T2.QUOTESYMBOL)
  union
  
  select * from  CRYPTO_DB.CCXT_PRO.CCXT_TOKEN_MAP where EXCHANGE='ascendex' and  QUOTESYMBOL='USD' and BASESYMBOL in
  (select distinct(T1.BASESYMBOL)
  from CRYPTO_DB.CCXT_PRO.CCXT_TOKEN_MAP as T1 inner join CRYPTO_DB.CCXT_PRO.CCXT_TOKEN_MAP as T2 
   on T1.BASESYMBOL=T2.BASESYMBOL and T1.EXCHANGE=T1.EXCHANGE where T1.EXCHANGE='ascendex' and T1.QUOTESYMBOL!=T2.QUOTESYMBOL
  ) """
fetched_data=conn1.cursor().execute(query1).fetch_pandas_all()
filtered_data=fetched_data[['PAIRS','TM_ID']].dropna(subset=['TM_ID'])
token_dic=dict(zip(filtered_data['PAIRS'], filtered_data['TM_ID']))
# with open('ascendex.txt','w') as convert_file:
#   convert_file.write(json.dumps(dic))
# OR
write_pandas(conn1, df=filtered_data, table_name="TOKENINFO", database="DEMO_DB", schema="PUBLIC")

In [ ]:



async def loop(exchange, symbol):
    while True:
        try:
            orderbook = await exchange.watch_ticker(symbol)
            print(exchange.iso8601(exchange.milliseconds()),"symbol:", orderbook['symbol'],"date::",orderbook['datetime'],"bid_price:",orderbook['bid'],"ask_price:",orderbook['ask'])
            #producer.send('ccxt_binanceus_watchticker',json.dumps(orderbook).encode('utf-8'),key=str(token_dict[orderbook["symbol"]]).encode('utf-8'))
        except Exception as e:
            print(str(e))

async def main():
    exchange = ccxtpro.binanceus({'enableRateLimit': True})
    #symbols = token_dict.keys() OR
    fetched_data1=conn1.cursor().execute("select PAIRS from TOKENINFO").fetch_pandas_all()
    symbols=fetched_data1.drop_duplicates().values.tolist()
    print(len(symbols))
    await asyncio.gather(*[loop(exchange, symbol) for symbol in symbols])
    await exchange.close()

In [ ]:
if __name__ == '__main__':
   # asyncio.run(main())
   await main()
